In [23]:
import pandas as pd
import re
import numpy as np

In [24]:
def load_data(input_file):
    data = pd.read_csv(input_file)
    return data

In [25]:
def inputcols(input_dataset):
    names = input_dataset['Name']
    details = input_dataset['Details']
    links = input_dataset['URL']
    return names, details , links

In [26]:
data = load_data('monitor.csv')
names, details , links = inputcols(data)

In [27]:
def extract_company_name(names,expression):
    companylist_name = []
    for name in names:
        re_search = re.search(expression, name)
        if re_search:
            companylist_name.append(name)
    return companylist_name

In [28]:
def extract_company_details(details,expression):
    companylist_det = []
    for det in details:
        re_search = re.search(expression, det)
        if re_search:
            companylist_det.append(det)
    return companylist_det;

In [29]:
def extract_company_links(input_dataset,expression):
    companylist_links = []
    for name, link in zip(input_dataset['Name'], input_dataset['URL']):
        re_search = re.search(expression, name)
        if re_search:
            if(link=='0'):
                print(name)
            companylist_links.append(link)
    return companylist_links;

In [30]:
def filtering_company_products(names, details,data, expression):
    companylist_name = extract_company_name(names, expression);
    companylist_det = extract_company_details(details, expression);
    companylist_links = extract_company_links(data, expression);
    companylist = pd.concat([pd.Series(companylist_name), pd.Series(companylist_det) , pd.Series(companylist_links)], axis=1)
    return companylist;

In [31]:
def extract_company_model_data(companylist,model_expression,company_name,search_grp):
    companymodelnumbers = []
    for name, det,link in companylist.itertuples(index=False):
        re_search = re.search(model_expression, name)
        if re_search:
            companymodelnumbers.append({"company": company_name, "name": name, "model": re_search.group(search_grp),"link":link})
        else:
            re_search2 = re.search(model_expression, det)
            if re_search2:
                companymodelnumbers.append({"company": company_name, "name": name, "model": re_search2.group(search_grp),"link":link})
            else:
                companymodelnumbers.append({"company": company_name, "name": name, "model": np.nan,"link":link})

    # NOW CHECK

    companymodelnumbers = pd.DataFrame(companymodelnumbers)
    return companymodelnumbers;

In [32]:
class Company:
    def __init__(self,expression,model_expression,names,details,links,company_name,search_grp):
        self.company_name = company_name
        self.names = names;
        self.details = details;
        self.links = links;
        self.expression = expression;
        self.model_expression = model_expression;
        self.search_grp = search_grp;
        self.productlist = pd.DataFrame()
        self.model_data = pd.DataFrame()
        
     
    def setcompanylist(self):
        self.productlist = filtering_company_products(self.names, self.details, data,self.expression)
    
    def setmodel_data(self):
        self.model_data = extract_company_model_data(self.productlist,self.model_expression,self.company_name,self.search_grp)
    
    def callall(self):
        self.setcompanylist()
        self.setmodel_data()
        
        return self.model_data

In [33]:
Lg = Company('LG ' , '\d{2}[MUBG][A-Za-z0-9]*(-B)?' , names , details ,links, "LG",0)
Hp = Company('(?i)HP ' , '(HP) (V?\d{1,2}[A-Za-z0-9]*) ' , names , details ,links, "HP",2)
samsung = Company('(?i)samsung ' , '(LC|LF|LS|LU)\w*(WXXL)?' , names , details ,links, "Samsung",0)
acer = Company('(?i)acer ' , '((?i)acer) (((?i)(nitro|aopen) )?[A-Za-z]+[\d+][A-Za-z0-9]*) ' , names , details ,links, "Acer",2)
dell = Company('(?i)dell ' , ' (D|E|P|SE|U)[0-9]*[A-Z0-9]* ' , names , details ,links, "Dell",0)
benq =  Company('(?i)benq ' , ' (PD|BL|EL|EW|EX|GW|PV|SW|XL|GL)[A-Z0-9]*' , names , details ,links, "Benq",0)
asus = Company('(?i)asus ' , ' (VG|VP|VZ|PA|XG|PB|VA)[A-Za-z0-9]* ' , names , details ,links, "Asus",0)
viewsonic = Company('(?i)viewsonic ' , ' (VA|VX|VP|XG|TD)[A-Za-z0-9]*' , names , details ,links, "Viewsonic",0)
gigabyte = Company('(?i)(gigabyte|aorus) ' , ' (G|M|CV|FI|AD)[A-Z0-9]* ' , names , details ,links, "Gigabyte",0)
msi = Company('(?i)msi ' , ' (G|MAG|MP)[A-Z0-9]* ' , names , details ,links, "MSI",0)
aoc = Company('(?i)aoc ' , ' (\()?(C|E|G)[A-Z0-9]+' , names , details ,links, "AOC",0)


Lg.callall()
Hp.callall()
samsung.callall()
acer.callall()
dell.callall()
benq.callall()
asus.callall()
viewsonic.callall()
gigabyte.callall()
msi.callall()
aoc.callall()


result = pd.concat([Lg.model_data,Hp.model_data,samsung.model_data,acer.model_data,dell.model_data,benq.model_data , asus.model_data , viewsonic.model_data , gigabyte.model_data , msi.model_data , aoc.model_data] ,axis=0, ignore_index=True)

result.to_csv('result.csv' , index=False)